<h1> Explore and create ML datasets </h1>

このノートブックでは、New Yorkでのタクシー乗車料金を予測する機械学習モデルを作成するために、その前段階として使用するデータの探索を行います。<br>

<div id="toc"></div>

では、必要なPythonライブラリのimportから始めていきましょう

In [ ]:
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil

<h3> BigQueryからサンプルデータを取得する</h3>


これから、<a href="https://console.cloud.google.com/bigquery?GK=nyc-tlc&page=table&t=trips&d=yellow&p=nyc-tlc">BigQueryのpublic dataset</a>を利用します。リンクをクリックして、カラム名をチェックしましょう。<br>
[Detail]タブに移動してレコード数が10億行以上あることを確認し、[Preview]タブでいくつかのデータを確認してみてください。


では、SQLを書いていくつかのフィールドをピックアップしてみましょう。

In [ ]:
sql = """
  SELECT
    pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude,
    dropoff_latitude, passenger_count, trip_distance, tolls_amount, 
    fare_amount, total_amount 
  FROM `nyc-tlc.yellow.trips`
  LIMIT 10
"""

In [ ]:
client = bigquery.Client()
trips = client.query(sql).to_dataframe()
trips

レコード数を増やしてグラフを書いてみましょう。返されるレコードの順番は保証されないため、LIMITの数を増やしてもどのレコードが返ってくるかはわかりません。<br>
適切なデータサンプルを取得するために、乗車時刻のHASHを利用して10万行に1行だけを取得しましょう。つまり、10億行のデータがあれば、およそ1万行のデータが取得できます。

In [ ]:
sql = """
  SELECT
    pickup_datetime,
    pickup_longitude, pickup_latitude, 
    dropoff_longitude, dropoff_latitude,
    passenger_count,
    trip_distance,
    tolls_amount,
    fare_amount,
    tip_amount,
    total_amount
  FROM
    `nyc-tlc.yellow.trips`
  WHERE
    MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
"""

In [ ]:
trips = client.query(sql).to_dataframe()
print('sample size: {}'.format(len(trips)))
trips[:10]

<h3> データを探索する</h3>

では、データを探索し、必要に応じてクリーンアップをしていきましょう。ここでは、PythonのSeabornパッケージを利用してグラフをビジュアライズし、Pandasを使ってスライシングとフィルタリングを行います。

In [ ]:
ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)
ax.figure.set_size_inches(10, 8)

なにかおかしな点があることに気づきますか？

どうやら、乗車距離が0であったり、乗車料金が明らかに外れ値であったりなどの無効なデータがたくさんあるようです。<br>
これらは分析対象から除外しましょう。BigQueryへのクエリを変更して、乗車距離が0マイルより大きく、かつ乗車料金が最低乗車料金（$2.50）以上のデータに絞り込むことができます。

追加されたWHERE句に注目してください。

In [ ]:
sql = """
  SELECT
    pickup_datetime,
    pickup_longitude, pickup_latitude, 
    dropoff_longitude, dropoff_latitude,
    passenger_count,
    trip_distance,
    tolls_amount,
    fare_amount,
    tip_amount,
    total_amount
  FROM
    `nyc-tlc.yellow.trips`
  WHERE
    MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
    AND trip_distance > 0 AND fare_amount >= 2.5
"""

In [ ]:
trips = client.query(sql).to_dataframe()
ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)
ax.figure.set_size_inches(10, 8)

45ドル、50ドル付近の直線は何でしょうか？これは、たとえばマンハッタンのJFK空港、La Guardia空港からの定額料金と推測できます。

では、データを確認して、値が何を意味しているのかを確認してみましょう。toll_amount（通行料金）、乗車料金(fare_amount)とtotal_amount（総額料金）の関係に注目してください。

In [ ]:
tollrides = trips[trips['tolls_amount'] > 0]
tollrides[tollrides['pickup_datetime'] == pd.Timestamp('2010-04-29 12:28:00', tz = 'UTC')]

上のいくつかのサンプルデータを見てみると、総額料金(total_amount)は乗車料金(fare_amount)と通行料金(tolls_amount）を反映していることが分かります。そして任意のチップ(tip_amount)が加えられていますが、チップを現金で支払った場合にはチップの値段はわかりません。
そのため、ここではfare_amount + tolls_amountを予測の対象として利用します。チップは乗客の裁量によるため、料金予測ツールからは除外するべきでしょう。

では、カラムごとの値の分布を確認しましょう。

In [ ]:
trips.describe()

どうやら経度(longitude)と緯度(latitude)の最小値と最大値に問題があるようです。

いくつかの履歴の乗車(pickup)位置と降車(dropoff)位置を見てみましょう。

In [ ]:
def showrides(df, numlines):
  lats = []
  lons = []
  for iter, row in df[:numlines].iterrows():
    lons.append(row['pickup_longitude'])
    lons.append(row['dropoff_longitude'])
    lons.append(None)
    lats.append(row['pickup_latitude'])
    lats.append(row['dropoff_latitude'])
    lats.append(None)

  sns.set_style("darkgrid")
  plt.figure(figsize=(10,8))
  plt.plot(lons, lats)

showrides(trips, 10)

In [ ]:
showrides(tollrides, 10)

予想どおり、通行料金(toll_amount)を含む乗車は他の一般的な乗車よりも距離が長いようです。

<h3> データクレンジングとその他の前処理</h3>

以下のデータクレンジングが必要です。

<ol>
<li>New Yorkの経度は-74前後、緯度は41前後のため、それ以外の乗降は除外する</li>
<li>乗客が0人のデータは除外する</li>
<li>総額料金(total_amount)を、乗車料金(fare_amount)と通行料金(tolls_amount)だけを反映するように変更する。また使用した2つの列は除外する</li>
<li>乗車時には、乗車位置と降車位置は分かっているが、乗車距離はまだ分からない（乗車距離はルートによって変わるため）。そのため乗車距離は機械学習のデータセットには含めない</li>
<li>タイムスタンプから曜日(dayofweek)と時間(hourofday)を生成し、タイムスタンプ自体は削除する</li>
<li>乗車人数自体はタクシーの料金と関係ないと考えられるため、乗車人数0のデータ削除の条件に使用した後、カラム自体は削除する。</li>
</ol>

距離0の乗車を除外したのと同様にBigQueryを使って前処理を行うことができますが、ここではPythonとPandasを使いましょう。<br>
本番環境では、リアルタイムの入力データに対して同様の前処理を行う必要があります。

入力データに対するこのような前処理は、機械学習を行う際に一般的です。

では、トレーニングに使用するデータを作成するために、WHERE句で絞り込む値を変更し、20万件程のデータを取得してクレンジングを適用しましょう<br>
（クエリには１分ほどの時間がかかります）

In [ ]:
sql = """
  SELECT
    pickup_datetime,
    pickup_longitude, pickup_latitude, 
    dropoff_longitude, dropoff_latitude,
    passenger_count,
    trip_distance,
    tolls_amount,
    fare_amount,
    tip_amount,
    total_amount
  FROM
    `nyc-tlc.yellow.trips`
  WHERE
    MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 5000) = 1
    AND trip_distance > 0 AND fare_amount >= 2.5
"""

In [ ]:
trips = client.query(sql).to_dataframe()

In [ ]:
def extract_dayofweek(t):
    return t.dayofweek

def extract_hour(t):
    return t.hour

def preprocess(trips_in):
    trips = trips_in.copy(deep=True)
    trips.fare_amount = trips.fare_amount + trips.tolls_amount
    trips['dayofweek'] = trips['pickup_datetime'].apply(extract_dayofweek)
    trips['hourofday'] = trips['pickup_datetime'].apply(extract_hour)
    del trips['tolls_amount']
    del trips['total_amount']
    del trips['tip_amount']
    del trips['trip_distance']
    del trips['pickup_datetime']
    qc = np.all([\
             trips['pickup_longitude'] > -78, \
             trips['pickup_longitude'] < -70, \
             trips['dropoff_longitude'] > -78, \
             trips['dropoff_longitude'] < -70, \
             trips['pickup_latitude'] > 37, \
             trips['pickup_latitude'] < 45, \
             trips['dropoff_latitude'] > 37, \
             trips['dropoff_latitude'] < 45, \
             trips['passenger_count'] > 0,
            ], axis=0)
    del trips['passenger_count']
    return trips[qc]

tripsqc = preprocess(trips)
tripsqc.describe()

データクレンジングにより、およそ5000行(222460 - 217514)、全体の2％ほどのデータが除外されました。これは良いバランスです。

では、機械学習用データセットを作成しましょう。

<h3> 機械学習用データセットの作成 </h3>

クレンジングされたデータを学習用と検証用、テスト用のデータセットにランダムに分割しましょう。

In [ ]:
shuffled = tripsqc.sample(frac=1)
trainsize = int(len(shuffled['fare_amount']) * 0.70)
validsize = int(len(shuffled['fare_amount']) * 0.15)

df_train = shuffled.iloc[:trainsize, :]
df_valid = shuffled.iloc[trainsize:(trainsize+validsize), :]
df_test = shuffled.iloc[(trainsize+validsize):, :]

In [ ]:
df_train.describe()

In [ ]:
df_valid.describe()

In [ ]:
df_test.describe()

3つのDataframeをcsvファイルに書き出しましょう。<br>

In [ ]:
!mkdir ../data

In [ ]:
def to_csv(df, filename):
    outdf = df.copy(deep=False)
    # reorder columns so that target is first column
    cols = ['fare_amount',
            'dayofweek',
            'hourofday',
            'pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
           ]
    outdf = outdf[cols]
    outdf.to_csv(filename, index_label=False, index=False)

path = '../data'
to_csv(df_train, '{}/taxi-train.csv'.format(path))
to_csv(df_valid, '{}/taxi-valid.csv'.format(path))
to_csv(df_test, '{}/taxi-test.csv'.format(path))

<h3> データセットが存在することを確認する </h3>

In [ ]:
!ls -l ../data/*.csv

学習、検証、テストに対応する３つのcsvファイルが作成されました。ファイルサイズの割合はデータの分割の割合に対応しています。

In [ ]:
%%bash
head ../data/taxi-train.csv

上手くいっていますね！機械学習用のデータセットが作成され、機械学習モデルのトレーニング、検証、評価の準備ができました。

Copyright 2016 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.